In [1]:
try:
    import elasticsearch
    from elasticsearch import Elasticsearch
    import pandas as pd
    import json
    from ast import literal_eval
    from tqdm import tqdm
    import datetime
    import os
    import sys
    import numpy as np
    from elasticsearch import helpers
    print('All libraries loaded successfully')
    
except Exception as e:
    print(e)

All libraries loaded successfully


In [23]:
# read the csv file
df = pd.read_csv('Test.csv')
print(df.shape)
df.head()

(12446, 52)


,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1227829.0,1006309.0,3168.0,121.0,3.0,1999.0,3688.0,Low,05-03-2012 00:00,580G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1227844.0,1022817.0,7271.0,121.0,3.0,1000.0,28555.0,High,05-10-2012 00:00,936,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1227847.0,1031560.0,22805.0,121.0,3.0,2004.0,6038.0,Medium,05-10-2012 00:00,EC210BLC,...,None or Unspecified,"9' 6""",Manual,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN
3,1227848.0,56204.0,1269.0,121.0,3.0,2006.0,8940.0,High,05-10-2012 00:00,330CL,...,None or Unspecified,None or Unspecified,Manual,Yes,Triple,NaN,NaN,NaN,NaN,NaN
4,1227863.0,1053887.0,22312.0,121.0,3.0,2005.0,2286.0,Low,05-10-2012 00:00,650K,...,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,None or Unspecified,NaN,NaN


In [3]:
# check for number of null values in each column
df.isna().sum()

SalesID                      7142
MachineID                    7142
ModelID                      7142
datasource                   7142
auctioneerID                 7142
YearMade                     7142
MachineHoursCurrentMeter    10566
UsageBand                   10861
saledate                     7142
fiModelDesc                  7142
fiBaseModel                  7142
fiSecondaryDesc              8255
fiModelSeries               11686
fiModelDescriptor           11072
ProductSize                  9814
fiProductClassDesc           7142
state                        7142
ProductGroup                 7142
ProductGroupDesc             7142
Drive_System                11081
Enclosure                    7143
Forks                        9839
Pad_Type                    11447
Ride_Control                10430
Stick                       11447
Transmission                10069
Turbocharged                11447
Blade_Extension             12080
Blade_Width                 12080
Enclosure_Type

In [7]:
df.columns

Index(['SalesID', 'MachineID', 'ModelID', 'datasource', 'auctioneerID',
       'YearMade', 'MachineHoursCurrentMeter', 'UsageBand', 'saledate',
       'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries',
       'fiModelDescriptor', 'ProductSize', 'fiProductClassDesc', 'state',
       'ProductGroup', 'ProductGroupDesc', 'Drive_System', 'Enclosure',
       'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission',
       'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type',
       'Engine_Horsepower', 'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier',
       'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System',
       'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type',
       'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer',
       'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls'],
      dtype='object')

In [24]:
# get selected columns
df_1 = df[['SalesID', 'MachineID', 'ModelID', 'datasource', 'auctioneerID']]
df_1 = df_1.dropna()
print(df_1.shape)
df_1.head()

(5304, 5)


,SalesID,MachineID,ModelID,datasource,auctioneerID
0,1227829.0,1006309.0,3168.0,121.0,3.0
1,1227844.0,1022817.0,7271.0,121.0,3.0
2,1227847.0,1031560.0,22805.0,121.0,3.0
3,1227848.0,56204.0,1269.0,121.0,3.0
4,1227863.0,1053887.0,22312.0,121.0,3.0


In [13]:
df_dict = df_1.to_dict('records')

In [14]:
df_dict

[{'SalesID': 1227829.0,
  'MachineID': 1006309.0,
  'ModelID': 3168.0,
  'datasource': 121.0,
  'auctioneerID': 3.0},
 {'SalesID': 1227844.0,
  'MachineID': 1022817.0,
  'ModelID': 7271.0,
  'datasource': 121.0,
  'auctioneerID': 3.0},
 {'SalesID': 1227847.0,
  'MachineID': 1031560.0,
  'ModelID': 22805.0,
  'datasource': 121.0,
  'auctioneerID': 3.0},
 {'SalesID': 1227848.0,
  'MachineID': 56204.0,
  'ModelID': 1269.0,
  'datasource': 121.0,
  'auctioneerID': 3.0},
 {'SalesID': 1227863.0,
  'MachineID': 1053887.0,
  'ModelID': 22312.0,
  'datasource': 121.0,
  'auctioneerID': 3.0},
 {'SalesID': 1227870.0,
  'MachineID': 1068694.0,
  'ModelID': 16747.0,
  'datasource': 121.0,
  'auctioneerID': 3.0},
 {'SalesID': 1227871.0,
  'MachineID': 1012262.0,
  'ModelID': 9133.0,
  'datasource': 121.0,
  'auctioneerID': 3.0},
 {'SalesID': 1227879.0,
  'MachineID': 1064696.0,
  'ModelID': 4699.0,
  'datasource': 121.0,
  'auctioneerID': 3.0},
 {'SalesID': 1227880.0,
  'MachineID': 1067175.0,
  'Mo

In [15]:
df_dict[0].keys()

dict_keys(['SalesID', 'MachineID', 'ModelID', 'datasource', 'auctioneerID'])

In [25]:
index_name = 'test_csv_elk'
def generator(index_name, df):
    for item, line in enumerate(df):
        yield{
            '_index': index_name,
            '_type': '_doc',
            '_id': line.get('SalesID', None),
            '_source': {
                'MachineID': line.get('MachineID', ''),
                'ModelID': line.get('ModelID', ''),
                'datasource': line.get('datasource', ''),
            }
        }
    raise StopIteration

In [26]:
# connect to elastic search
web_address = 'http://localhost:9200/'
es = Elasticsearch(timeout = 600, hosts = web_address)
es.ping()

True

In [27]:
settings = {
    'settings':{
        'number_of_shards':1,
        'number_of_replicas':0
    },
    'mappings':{
        'properties':{
            'MachineID':{
                'type': 'float'
                
            },
            'ModelID':{
                'type': 'float'
            }
            
        }
    }
}

In [28]:
my = es.indices.create(index = index_name, 
                       ignore = [404, 400], 
                       body = settings)

In [31]:
try:
    res = helpers.bulk(client = es, 
                       actions = generator(index_name, df_dict))
    print('completed')
except Exception as e:
    print(e)
    pass

C:\Users\Arjun Janamatti\Anaconda3\envs\tf_nptel\lib\site-packages\elasticsearch\connection\base.py:193: ElasticsearchDeprecationWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


completed
